# 텐서플로우를 이용한 신경망 만들기 예제

여기에서는 간단한 신경망을 구축하고 학습시킴으로써, MNIST라는 손글씨를 얼마만큼 정확하게 맞추는 지 확인해 보도록 하겠습니다.

### 1. 필요한 모듈들을 불러옵니다.

텐서플로우 라이브러리와 numpy 라이브러리, 그리고 matplotlib.pyplot을 불러 옵니다.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

MNIST란 손으로 쓴 0~9의 숫자 모음 데이터입니다. 
* 텐서플로우에는 예제로서 mnist가 포함되어 있습니다. 
* mnist 데이터를 다루기 편리하도록 몇가지 도구를 제공하므로 우리는 이를 활용하도록 하겠습니다. 

### 2. MNIST 데이터를 다운로드합니다.

input_data를 불러온 다음에, 이를 이용하여 mnist 데이터를 다운로드 받습니다. 
* one_hot을 True로 설정해 줍니다. 
* 예를 들어 one_hot 방식으로 3은 (0,0,0,1,0,0,0,0,0,0)로, 7은 (0,0,0,0,0,0,0,1,0,0)로 표현됩니다. 

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/mnist/", one_hot=True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


### 3. MNIST 데이터 살펴보기

MNIST 데이터는 training data, validation data, test data로 구분되어 있으며 각각은 image와 label의 쌍으로 구성되어 있습니다. 

Training data의 갯수는 55,000개입니다.

In [3]:
print len(mnist.train.labels)
print len(mnist.train.images)

55000
55000


Validation data의 갯수는 5,000개, Test data의 갯수는 10,000개입니다.

In [4]:
print mnist.validation.num_examples
print mnist.test.num_examples

5000
10000


One_hot 방식으로 불러온 mnist 일부를 보면 아래와 같습니다.

In [5]:
mnist.test.labels[0:5, :]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]])

이를 일반적인 숫자로 변환하면 아래와 같습니다.

In [6]:
mnist.test.cls = np.argmax(mnist.test.labels, axis=1)
mnist.test.cls[0:5]

array([7, 2, 1, 0, 4])

### 4. 하이퍼 파라미터 설정해주기

In [7]:
num_epochs = 30
learning_rate = 0.01

우리는 입력 계층 --> 은닉 계층 1 --> 은닉 계층 2 --> 출력 계층으로 구성된 신경망을 구축할 것입니다. 각 계층에서의 노드의 갯수는 아래와 같이 설정하도록 하겠습니다.
* 입력 계층의 노드 수 : 784
* 은닉 계층1의 노드 수 : 256
* 은닉 계층2의 노드 수 : 256
* 출력 계층의 노드 수 : 10

* 입력 계층 : MNIST 이미지 각각은 28x28의 해상도를 가지고 있기 때문에 입력 계층의 노드수는 784가 됩니다.
* 출력 계층 : 0부터 9까지 총 10개의 클래스로 구성 (One_hot) 되기 때문에 출력 계층의 노드 수는 10이 됩니다.

In [8]:
num_node_input = 28*28
num_node_hidden1 = 256
num_node_hidden2 = 256
num_node_output = 10

### 5. 모델 만들기

입력 계층을 플레이스홀더로 정의해 줍니다.

In [9]:
x_true = tf.placeholder(tf.float32, [None, num_node_input])
y_true = tf.placeholder(tf.float32, [None, num_node_output])

은닉 계층 1의 가중치와 편향을 설정해 줍니다.

In [10]:
weight_1 = tf.Variable(tf.truncated_normal([num_node_input, num_node_hidden1], stddev=0.01))
bias_1 = tf.Variable(tf.zeros([num_node_hidden1]))

은닉 계층 2의 가중치와 편향을 설정해 줍니다.

In [11]:
weight_2 = tf.Variable(tf.truncated_normal([num_node_hidden1, num_node_hidden2], stddev=0.01))
bias_2 = tf.Variable(tf.zeros([num_node_hidden2]))

출닉 계층의 가중치와 편향을 설정해 줍니다.

In [12]:
weight_3 = tf.Variable(tf.truncated_normal([num_node_hidden2, num_node_output], stddev=0.01))
bias_3 = tf.Variable(tf.zeros([num_node_output]))

은닉 계층 1, 은닉 계층 2, 출력 계층에서 연산을 수행하고 relu를 이용해 non-linearity를 주게 됩니다.

In [13]:
hidden_1 = tf.nn.relu(tf.add(tf.matmul(x_true, weight_1), bias_1))
hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, weight_2), bias_2))
y_pred = tf.nn.relu(tf.add(tf.matmul(hidden_2, weight_3), bias_3))

예측 값과 실제 값의 차를 이용해 cross entropy를 구해 주고 이를 평균한 값이 비용이 됩니다. 비용을 최소화하도록 학습시켜 주기 위해 optimizer를 정의해 줍니다. 

In [14]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

### 6. 학습시키기

세션을 실행합니다.

In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

MNIST의 training data는 55,000개나 되기 때문에 batch로 묶어 주어 피드해 주게 됩니다.

In [16]:
batch_size = 100
total_batch = int(mnist.train.num_examples/batch_size)

학습을 시작합니다. 
* 앞에서 정의한 num_epochs 만큼 학습을 진행합니다.
* batch로 100개씩 묶어서 피드하게 됩니다.
* 텐서플로우에서는 mnist 데이터의 피드를 편하게 하기 위해 next_batch()라는 함수를 제공합니다.

In [17]:
for epoch in range(num_epochs):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, {x_true:batch_xs, y_true:batch_ys})
        total_cost += sess.run(cost, {x_true:batch_xs, y_true:batch_ys})
    
    print "Epoch : {%04d}" % (epoch + 1), "Cost : {:.3f}".format(total_cost / total_batch)
    
print "최적화가 완료되었습니다."

Epoch : {0001} Cost : 1.914
Epoch : {0002} Cost : 1.873
Epoch : {0003} Cost : 1.867
Epoch : {0004} Cost : 1.828
Epoch : {0005} Cost : 1.610
Epoch : {0006} Cost : 1.409
Epoch : {0007} Cost : 1.396
Epoch : {0008} Cost : 1.392
Epoch : {0009} Cost : 1.386
Epoch : {0010} Cost : 1.386
Epoch : {0011} Cost : 1.381
Epoch : {0012} Cost : 1.378
Epoch : {0013} Cost : 1.378
Epoch : {0014} Cost : 1.377
Epoch : {0015} Cost : 1.378
Epoch : {0016} Cost : 1.375
Epoch : {0017} Cost : 1.374
Epoch : {0018} Cost : 1.373
Epoch : {0019} Cost : 1.373
Epoch : {0020} Cost : 1.372
Epoch : {0021} Cost : 1.370
Epoch : {0022} Cost : 1.370
Epoch : {0023} Cost : 1.369
Epoch : {0024} Cost : 1.370
Epoch : {0025} Cost : 1.370
Epoch : {0026} Cost : 1.367
Epoch : {0027} Cost : 1.369
Epoch : {0028} Cost : 1.371
Epoch : {0029} Cost : 1.367
Epoch : {0030} Cost : 1.367
최적화가 완료되었습니다.


### 7. 평가하기

* 예측 값과 실제 값이 동일한 경우를 카운트해 줍니다. 
* 이를 실수로 캐스팅한 다음에 평균을 냄으로써 정확도를 정의합니다. 
* 세션을 수행하여 테스트 이미지와 레이블을 전달하여 정확도를 구할 수 있습니다. 

In [19]:
correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print "정확도 :", sess.run(accuracy, {x_true: mnist.test.images,
                                     y_true: mnist.test.labels})

정확도 : 0.4986
